# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [75]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [76]:
data.head()


,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [78]:


customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

customer_products.head()


,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [79]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [80]:

def simmiliraty_matrix(x): 
    distances = pd.DataFrame(1/(1 + squareform(pdist(x.transpose()))), 
                         index=x.columns, columns=x.columns) 
    return distances

similar = simmiliraty_matrix(prod_cust_pivot)

similar.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [81]:
similar = 1 / (1 + similar)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [82]:
similar = simmiliraty_matrix(prod_cust_pivot)
similar[33].sort_values(ascending = False).head(6)


CustomerID
33      1.000000
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [87]:
records = list(similar[33].sort_values(ascending = False).head(6).index)

records.pop(0) #removes 33

sim = customer_products[customer_products['CustomerID'].isin(records)] 

sim.head()
#

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [88]:
step_6 = sim.pivot_table(values='Quantity', 
                columns='CustomerID', 
                index='ProductName', 
                aggfunc='sum').fillna(0)

step_6['Total'] = step_6.sum(axis = 1)


step_6 = step_6.sort_values('Total', ascending=False)

step_6.head()

CustomerID,264,2503,3305,3317,3535,Total
ProductName,,,,,,
Butter - Unsalted,0.0,2.0,0.0,1.0,0.0,3.0
Wine - Ej Gallo Sierra Valley,0.0,0.0,2.0,1.0,0.0,3.0
Towels - Paper / Kraft,1.0,0.0,1.0,0.0,1.0,3.0
Soup - Campbells Bean Medley,1.0,0.0,0.0,1.0,1.0,3.0
Wine - Blue Nun Qualitatswein,1.0,0.0,1.0,0.0,1.0,3.0


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [89]:
not_33 = list(prod_cust_pivot[prod_cust_pivot[33] == 0].index)

recomm = step_6[step_6.index.isin(not_33)].sort_values('Total', ascending = False).head(5)

recomm


CustomerID,264,2503,3305,3317,3535,Total
ProductName,,,,,,
Butter - Unsalted,0.0,2.0,0.0,1.0,0.0,3.0
Soup - Campbells Bean Medley,1.0,0.0,0.0,1.0,1.0,3.0
Wine - Blue Nun Qualitatswein,1.0,0.0,1.0,0.0,1.0,3.0
Wine - Ej Gallo Sierra Valley,0.0,0.0,2.0,1.0,0.0,3.0
Beans - Wax,0.0,0.0,2.0,0.0,0.0,2.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
cust_prod_pivot = prod_cust_pivot.T

prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)

cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

recommendations = {}

customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [92]:
user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [102]:
cust_dist_last = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'jaccard'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)
cust_dist_last.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.0,0.5,0.5,0.5,0.5,0.5,0.500000,0.5,0.5,0.500000,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.500000
200,0.5,1.0,0.5,0.5,0.5,0.5,0.520325,0.5,0.5,0.500000,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.500000
264,0.5,0.5,1.0,0.5,0.5,0.5,0.500000,0.5,0.5,0.500000,...,0.5,0.515556,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.500000
356,0.5,0.5,0.5,1.0,0.5,0.5,0.500000,0.5,0.5,0.513393,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.500000
412,0.5,0.5,0.5,0.5,1.0,0.5,0.500000,0.5,0.5,0.500000,...,0.5,0.500000,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.530973


In [103]:
cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015


In [98]:
# simmiliraty_matrix(cust_prod_pivot)